In [ ]:
import os
import random
from PIL import Image
from albumentations import (
    HorizontalFlip, VerticalFlip, RandomRotate90, ShiftScaleRotate, 
    RandomBrightnessContrast, HueSaturationValue, Resize, Compose
)
from albumentations.core.composition import OneOf

In [ ]:
input_dir = "./cleaned_dataset"  
output_dir = "./augmented_dataset"  
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def get_augmentation_pipeline():
    return Compose([
        HorizontalFlip(p=0.5),  # Randomly flip images horizontally
        VerticalFlip(p=0.5),    # Randomly flip images vertically
        RandomRotate90(p=0.5),  # Rotate by 90 degrees randomly
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
        HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.7),
        Resize(256, 256)  # Resize all images to 256x256 pixels
    ])

In [ ]:
def augment_images_for_soil_type(soil_type_dir, output_type_dir, num_augmentations=5):
    os.makedirs(output_type_dir, exist_ok=True)
    
    # Load augmentation pipeline
    augment_pipeline = get_augmentation_pipeline()
    
    # Process each image in the soil type directory
    for filename in os.listdir(soil_type_dir):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        image_path = os.path.join(soil_type_dir, filename)
        image = Image.open(image_path).convert("RGB")  # Open image and ensure RGB
        
        for i in range(num_augmentations):
            # Apply augmentation
            image_array = np.array(image)
            augmented = augment_pipeline(image=image_array)
            
            # Save augmented image
            augmented_image = Image.fromarray(augmented['image'])
            augmented_filename = f"{os.path.splitext(filename)[0]}_aug_{i+1}.jpg"
            augmented_image.save(os.path.join(output_type_dir, augmented_filename))


In [ ]:
def augment_dataset(input_dir, output_dir, num_augmentations=5):
    for soil_type in os.listdir(input_dir):
        soil_type_dir = os.path.join(input_dir, soil_type)
        output_type_dir = os.path.join(output_dir, soil_type)
        
        if os.path.isdir(soil_type_dir):
            print(f"Augmenting images for soil type: {soil_type}")
            augment_images_for_soil_type(soil_type_dir, output_type_dir, num_augmentations)

In [ ]:
augment_dataset(input_dir, output_dir, num_augmentations=5)
print("Data augmentation completed!")